In [1]:
# based SUMMA-mizuroute settings, create settings for CTSM
# copy files and change a few settings

In [14]:
import pandas as pd
import numpy as np
import os, sys, glob, shutil
import matplotlib.pyplot as plt
import xarray as xr


def change_text_value(file, newsettings, separator, comment):
    # file: target file
    # start: start string (note for mizuroute.control file, this should be "<name" because mizuroute settings are within "<>")
    # newsettings: dict contain target variables and values
    # separator: character separating variables and values (e.g., "=" for variable=value)
    # comment: character used as character

    # separator, and comment are '\'', and ! for summa fileManager.txt
    # separator, and comment are ' ', and ! for mizuroute control file
    # separator, and comment are ' ', and # for ostIn.txt
    # separator, and comment are '=', and # for run_trial.sh

    # example:
    # summa_setting = {}
    # summa_setting['settingsPath'] = '/the/path/'
    # summa_setting['simStartTime'] = '2009-07-01 00:00'
    # change_text_value('summa_fileManager.txt', summa_setting, separator='\'', comment='!')

    if (len(newsettings) > 0) and os.path.isfile(file):
        # read raw data
        with open(file) as f:
            contents = f.readlines()
        # save a new file
        file_new = file + '-temp'
        with open(file_new, 'w') as f:
            for line in contents:
                for name, value in newsettings.items():
                    if line.startswith(name):
                        line2 = line.split(comment)[0].strip()
                        if line2.count(separator) == 2: # format: xxx_sep_value_sep (only summa fileManager.txt)
                            oldvalue = line2.split(separator)[1].strip()
                        else:
                            oldvalue = line2.split(separator)[-1].strip()
                        if not isinstance(value, str):
                            value = str(value)
                        line = line.replace(oldvalue, value)
                f.write(line)
        # replace old file
        os.remove(file)
        shutil.move(file_new, file)


In [15]:
file = '/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv'
df_info = pd.read_csv(file)

In [16]:
inpath = ' /glade/campaign/cgd/tss/people/mozhgana/projects/routing/camels'
outpath = '/glade/work/guoqiang/CTSM_CAMELS/mizuroute_settings'

In [20]:
for i in range(len(df_info)):
    basinid = df_info.iloc[i]['hru_id']
    inpathi = f'{inpath}/{basinid:08}/route_settings/'
    outpathi = f'{outpath}/level1_{i}/'
    os.makedirs(outpathi, exist_ok=True)
    os.system(f'cp {inpathi}/* {outpathi}')
    os.system(f'cp param.nml.default {outpathi}')
    os.system(f'cp control.txt {outpathi}')

    outfile = f'{outpathi}/control.txt'

    newsettings = {'<ancil_dir>': outpathi, 
                   '<input_dir>': 'to_be_changed', 
                   '<output_dir>': 'to_be_changed',
                   '<fname_ntopOld>': f'ntopo_MERIT_Hydro_v1_{basinid:08}.nc', 
                  }
    change_text_value(outfile, newsettings, ' ', '!')